# Study project: Evaluation of different Sim-2-Real approaches

## Analysis of results

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import json
import PIL.Image as Image
import io

path = 'C:/Users/janni/AppData/LocalLow/DefaultCompany/SP-Sim2Real/BaseLevel_Epochs1_CurriculumLearning_Run1/5100.json'

with open(path) as file:
    data = json.load(file)

print('ActionM1:', data['actions'][0])
print('ActionM2:', data['actions'][1])
image = Image.open(io.BytesIO(bytearray(data['image'])))
image.save('C:/Users/janni/AppData/LocalLow/DefaultCompany/SP-Sim2Real/BaseLevel_Epochs1_CurriculumLearning_Run1/5100.png')

ActionM1: 0.21679267287254333
ActionM2: -0.2979181408882141


## Create plots from tensorboard data

In [15]:
import os
import numpy as np

resultDirs = os.listdir('Results')

for dir in resultDirs:
    results = os.listdir('Results/' + dir)

    diagrams = [
        'BatteryCapacity', 'BatteryFailure', 'Beta', 'BoundaryFailure', 'CumulativeReward', 'DistanceTargetDropoff',
        'Entropy', 'EpisodeLength', 'Epsilon', 'ExtrinsicReward', 'ExtrinsicValueEstimate', 'LearningRate', 'LessonNumber',
        'MaximalEnergyCollected', 'PolicyLoss', 'TargetCollected', 'ValueLoss'
    ]

    titles = [
        'Battery capacity after collection', 'Failure due to battery', 'Beta', 'Failure due to boundary exceeding', 'Cumulative Reward', 'Distance Factor',
        'Entropy', 'Steps per Episode', 'Epsilon', 'Extrinsic Reward', 'Extrinsic Value Estimate', 'Learning Rate', 'Lesson',
        'Maximal Energy Collected', 'Policy Loss', 'Collected Targets', 'Value Loss'
    ]

    yAxis = [
        'Battery Capacity', 'Failures', 'Beta', 'Failures', 'Reward', 'Factor',
        'Entropy', 'Length', 'Epsilon', 'Reward', 'Estimate', 'Rate', 'Lesson',
        'Energy Collected', 'Loss', 'Collections', 'Loss'
    ]

    for idx, diagram in enumerate(diagrams):
        paths = []
        for result in results:
            paths.append('./Results/' + dir + '/' + result + '/' + diagram + ".csv")

        data = []
        for i in range(0, len(paths)):
            d = np.genfromtxt(paths[i], delimiter=',')[1:, 1:]
            data.append(d)

        import matplotlib.pyplot as plt
        import itertools

        smoothing = 2

        colors = itertools.cycle(('r-', 'g-', 'b-', 'y-')) 
        for i in range(0, len(data)):
            d = data[i]

            x = d[:,0]
            y = d[:,1]

            if smoothing > 0: 
                x = np.delete(x, np.arange(0, x.size, smoothing))
                y = np.delete(y, np.arange(0, y.size, smoothing))

            plt.plot(x, y, next(colors))

        plt.title(titles[idx])
        plt.xlabel('Steps')
        plt.ylabel(yAxis[idx])
        plt.grid(color='gray', alpha=0.5, linestyle='dashed', linewidth=0.5)

        if (diagram == 'CumulativeReward'):
            plt.legend(results, loc="upper left", fontsize="small")

        # Source: https://matplotlib.org/stable/tutorials/introductory/usage.html

        # Save plot 
        if (not os.path.isdir('./Plots/' + dir)): os.mkdir('./Plots/' + dir)
        print('Created new plot: ./Plots/' + dir + '/' + diagram + '.pdf')
        plt.savefig('./Plots/' + dir + '/' + diagram + '.pdf', format='pdf', bbox_inches='tight', dpi=1200) 
        plt.clf()

Created new plot: ./Plots/levels/BatteryCapacity.pdf
Created new plot: ./Plots/levels/BatteryFailure.pdf
Created new plot: ./Plots/levels/Beta.pdf
Created new plot: ./Plots/levels/BoundaryFailure.pdf
Created new plot: ./Plots/levels/CumulativeReward.pdf
Created new plot: ./Plots/levels/DistanceTargetDropoff.pdf
Created new plot: ./Plots/levels/Entropy.pdf
Created new plot: ./Plots/levels/EpisodeLength.pdf
Created new plot: ./Plots/levels/Epsilon.pdf
Created new plot: ./Plots/levels/ExtrinsicReward.pdf
Created new plot: ./Plots/levels/ExtrinsicValueEstimate.pdf
Created new plot: ./Plots/levels/LearningRate.pdf
Created new plot: ./Plots/levels/LessonNumber.pdf
Created new plot: ./Plots/levels/MaximalEnergyCollected.pdf
Created new plot: ./Plots/levels/PolicyLoss.pdf
Created new plot: ./Plots/levels/TargetCollected.pdf
Created new plot: ./Plots/levels/ValueLoss.pdf


<Figure size 432x288 with 0 Axes>

In [2]:
import os
import numpy as np

resultDirs = os.listdir('Results')

for dir in resultDirs:
    results = os.listdir('Results/' + dir)

    diagrams = [
        'BatteryCapacity', 'BatteryFailure', 'Beta', 'BoundaryFailure', 'CumulativeReward', 'DistanceTargetDropoff',
        'Entropy', 'EpisodeLength', 'Epsilon', 'ExtrinsicReward', 'ExtrinsicValueEstimate', 'LearningRate', 'LessonNumber',
        'MaximalEnergyCollected', 'PolicyLoss', 'TargetCollected', 'ValueLoss'
    ]


    for idx, diagram in enumerate(diagrams):
        paths = []
        for result in results:
            paths.append('./Results/' + dir + '/' + result + '/' + diagram + ".csv")

        data = []
        for i in range(0, len(paths)):
            d = np.genfromtxt(paths[i], delimiter=',')[1:, 1:]
            data.append(d)

        import matplotlib.pyplot as plt
        import itertools

        smoothing = 2

        colors = itertools.cycle(('r-', 'g-', 'b-')) 
        for i in range(0, len(data)):
            d = data[i]

            x = d[:,0]
            y = d[:,1]

            if smoothing > 0: 
                x = np.delete(x, np.arange(0, x.size, smoothing))
                y = np.delete(y, np.arange(0, y.size, smoothing))

            if dir == 'lambda':
                if (diagram == 'BoundaryFailure') | (diagram == 'BatteryFailure'):
                    last = None
                    for xi in range(1, x.shape[0]):
                        if y[xi-1]-20 > y[xi]:
                            print("Found in", diagram)
                            y[xi:] = y[xi:] + y[xi-1]
                            print(str(y[xi-1]) + " : " + str(y[xi]))
                            last = None

            if dir == 'lambda':
                if (diagram == 'TargetCollected'):
                    last = None
                    for xi in range(1, x.shape[0]):
                        if y[xi-1]-1000 > y[xi]:
                            print("Found in", diagram)
                            y[xi:] = y[xi:] + y[xi-1]
                            print(str(y[xi-1]) + " : " + str(y[xi]))
                            last = None

            plt.plot(x, y, next(colors))

        plt.title(titles[idx])
        plt.xlabel('Steps')
        plt.ylabel(yAxis[idx])
        plt.grid(color='gray', alpha=0.5, linestyle='dashed', linewidth=0.5)

        if (diagram == 'CumulativeReward'):
            plt.legend(results, loc="upper left", fontsize="small")

        # Source: https://matplotlib.org/stable/tutorials/introductory/usage.html

        # Save plot 
        if (not os.path.isdir('./Plots/' + dir)): os.mkdir('./Plots/' + dir)
        print('Created new plot: ./Plots/' + dir + '/' + diagram + '.pdf')
        plt.savefig('./Plots/' + dir + '/' + diagram + '.pdf', format='pdf', bbox_inches='tight', dpi=1200) 
        plt.clf()

Created new plot: ./Plots/base/BatteryCapacity.pdf
Created new plot: ./Plots/base/BatteryFailure.pdf
Created new plot: ./Plots/base/Beta.pdf
Created new plot: ./Plots/base/BoundaryFailure.pdf
Created new plot: ./Plots/base/CumulativeReward.pdf
Created new plot: ./Plots/base/DistanceTargetDropoff.pdf
Created new plot: ./Plots/base/Entropy.pdf
Created new plot: ./Plots/base/EpisodeLength.pdf
Created new plot: ./Plots/base/Epsilon.pdf
Created new plot: ./Plots/base/ExtrinsicReward.pdf
Created new plot: ./Plots/base/ExtrinsicValueEstimate.pdf
Created new plot: ./Plots/base/LearningRate.pdf
Created new plot: ./Plots/base/LessonNumber.pdf
Created new plot: ./Plots/base/MaximalEnergyCollected.pdf
Created new plot: ./Plots/base/PolicyLoss.pdf
Created new plot: ./Plots/base/TargetCollected.pdf
Created new plot: ./Plots/base/ValueLoss.pdf
Created new plot: ./Plots/cnn/BatteryCapacity.pdf
Created new plot: ./Plots/cnn/BatteryFailure.pdf
Created new plot: ./Plots/cnn/Beta.pdf
Created new plot: ./P

<Figure size 432x288 with 0 Axes>